In [21]:
# from requests import get
import requests
import re
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
import random
import numpy as np, pandas as pd
import seaborn as sns
import urllib.request
from tqdm import tqdm

In [22]:
# Test
url = "https://en.wikipedia.org/wiki/Asomiya_Khabar"
response = requests.get(url=url)

soup = BeautifulSoup(response.content, 'html.parser')
des_num = len(soup.find_all("p"))
print(des_num)
description_unfiltered = str(soup.find_all("p")[0])
description = re.sub(r"<.*?>", "", description_unfiltered)
print(description_unfiltered)
print(description)
if description == str("\n"):
    description_unfiltered = str(soup.find_all("p")[1])
    description = re.sub(r"<.*?>", "", description_unfiltered)
    print(description_unfiltered)
    print(description)
    
allLinks = soup.find(id="bodyContent").find_all("a")
print(allLinks[0]["href"])

3
<p class="mw-empty-elt">
</p>


<p><b><span title="Assamese-language text"><i lang="as-Latn">Asomiya Khabar</i></span></b> (<a href="/wiki/Assamese_language" title="Assamese language">Assamese:</a> অসমীয়া খবৰ) (or <i><b>Oxomiya Khobor</b></i>) is an <a href="/wiki/Assamese_language" title="Assamese language">Assamese</a> daily newspaper published simultaneously from <a href="/wiki/Guwahati" title="Guwahati">Guwahati</a> and <a href="/wiki/Jorhat" title="Jorhat">Jorhat</a>.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> It is one of the highest circulated Assamese daily newspaper. A weekly supplement named <i>Deoboriya Khabar</i> is published with it on every Sunday. It was placed as the second largest vernacular daily of the region.<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup> Veteran journalist Shankar Laskar is in the position of editor of <span title="Assamese-language text"><i lang="as-Latn">Asomiya Khabar</i></span> and 

In [23]:
Dont_include_in_links = r"User|File\:|wiki\/Wikipedia\:|Special\:|Help\:|\#|Template|Category\:"

def Scraper(url, data_frame):
    try:
        response = requests.get(url=url)
    except:
        sleep(5)
        print("Sleeping...")
        response = requests.get(url=url)

    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find(id="firstHeading")
    try:
        description_unfiltered = str(soup.find_all("p")[0])
        # Make description readable
        description = re.sub(r"<.*?>", "", description_unfiltered)
    except:
        print("Cannot find paragraph?")
        description = title.text
    
    if description == str("\n"):
        try:
            description_unfiltered = str(soup.find_all("p")[1])
            description = re.sub(r"<.*?>", "", description_unfiltered)
        except:
            print("Cannot find paragraph??")
            description = title.text
            
    df_temp = pd.DataFrame([[title.text, description, url]], columns=["Title", "Description", "Link"])
    data_result = pd.concat([data_frame, df_temp], ignore_index=True)

    # Randomly get a next link
    allLinks = soup.find(id="bodyContent").find_all("a")
    random.shuffle(allLinks)
    linkToScrape = 0
    
    for link in allLinks:
        # Only interested in other wiki articles
        try:
            if link['href'].find("/wiki/") == -1 or \
                len(re.findall(Dont_include_in_links, link['href'])) >= 1: #or link['href'].find("\."): 
                continue
        except:
            print("Error! Check next link...")
            continue

        # Use this link to scrape
        linkToScrape = link
        break

    # The next link to scrape
    next_link = "https://en.wikipedia.org" + linkToScrape['href']
    # The link should be valid
    next_link = re.sub(r"^https.*https", "https", next_link)
    # print(title.text)
    # print(next_link)
    
    return data_result, next_link

In [24]:
df = pd.read_csv("data.csv", index_col=False)
next_link = "https://en.wikipedia.org/wiki/Hans_B%C3%B8rli"
display(df)

,Title,Description,Link
0,Web scraping,"Web scraping, web harvesting, or web data extr...",https://en.wikipedia.org/wiki/Web_scraping
1,Wrapper (data mining),Wrapper in data mining is a procedure that ext...,https://en.wikipedia.org/wiki/Wrapper_(data_mi...
2,Microformat,Microformats (μF)[note 1] are a set of defined...,https://en.wikipedia.org/wiki/Microformat
3,Common Logic,Common Logic (CL) is a framework for a family ...,https://en.wikipedia.org/wiki/Common_Logic
4,ISLISP,ISLISP (also capitalized as ISLisp) is a progr...,https://en.wikipedia.org/wiki/ISLISP
...,...,...,...
21366,MacArthur Fellows Program,"The MacArthur Fellows Program, also known as t...",https://en.wikipedia.org/wiki/MacArthur_Fellow...
21367,Guillermo Algaze,"Guillermo Algaze (born November 24, 1954) is a...",https://en.wikipedia.org/wiki/Guillermo_Algaze
21368,University of Puerto Rico,The University of Puerto Rico (Spanish: Univer...,https://en.wikipedia.org/wiki/University_of_Pu...
21369,El Vocero,El Vocero de Puerto Rico is a Puerto Rican fre...,https://en.wikipedia.org/wiki/El_Vocero


In [25]:
next_link = "https://en.wikipedia.org/wiki/Google_Checkout"

In [26]:
for i in tqdm(range(0, 1000)):
    df, next_link = Scraper(next_link, df)
    
display(df)
print(next_link)

  6%|▋         | 63/1000 [00:32<06:24,  2.44it/s]

Error! Check next link...


 11%|█         | 111/1000 [01:07<06:01,  2.46it/s]

Error! Check next link...


 57%|█████▋    | 569/1000 [04:53<02:10,  3.30it/s]

...

 83%|████████▎ | 831/1000 [07:30<02:29,  1.13it/s]

Error! Check next link...


100%|██████████| 1000/1000 [09:04<00:00,  1.84it/s]


,Title,Description,Link
0,Web scraping,"Web scraping, web harvesting, or web data extr...",https://en.wikipedia.org/wiki/Web_scraping
1,Wrapper (data mining),Wrapper in data mining is a procedure that ext...,https://en.wikipedia.org/wiki/Wrapper_(data_mi...
2,Microformat,Microformats (μF)[note 1] are a set of defined...,https://en.wikipedia.org/wiki/Microformat
3,Common Logic,Common Logic (CL) is a framework for a family ...,https://en.wikipedia.org/wiki/Common_Logic
4,ISLISP,ISLISP (also capitalized as ISLisp) is a progr...,https://en.wikipedia.org/wiki/ISLISP
...,...,...,...
22366,Chemical Markup Language,Chemical Markup Language (ChemML or CML) is an...,https://en.wikipedia.org/wiki/Chemical_Markup_...
22367,PubMed,PubMed is a free database including primarily ...,https://en.wikipedia.org/wiki/PMID_(identifier)
22368,Digital object identifier,A digital object identifier (DOI) is a persist...,https://en.wikipedia.org/wiki/Doi_(identifier)
22369,ISO/IEC 7811,ISO/IEC 7811 Identification cards — Recording ...,https://en.wikipedia.org/wiki/ISO/IEC_7811


https://en.wikipedia.org/wiki/Richard_Wiseman


In [27]:
df = df.drop_duplicates(subset=["Title"])
display(df)
print(next_link)

,Title,Description,Link
0,Web scraping,"Web scraping, web harvesting, or web data extr...",https://en.wikipedia.org/wiki/Web_scraping
1,Wrapper (data mining),Wrapper in data mining is a procedure that ext...,https://en.wikipedia.org/wiki/Wrapper_(data_mi...
2,Microformat,Microformats (μF)[note 1] are a set of defined...,https://en.wikipedia.org/wiki/Microformat
3,Common Logic,Common Logic (CL) is a framework for a family ...,https://en.wikipedia.org/wiki/Common_Logic
4,ISLISP,ISLISP (also capitalized as ISLisp) is a progr...,https://en.wikipedia.org/wiki/ISLISP
...,...,...,...
22355,Filter (software),A filter is a computer program or subroutine t...,https://en.wikipedia.org/wiki/Filter_(Unix)
22359,Name collision,"In computer programming, a name collision is t...",https://en.wikipedia.org/wiki/Name_collision
22363,WebXR,WebXR Device API is a Web application programm...,https://en.wikipedia.org/wiki/WebXR
22364,WebPlatform.org,WebPlatform.org (or WebPlatform) was a communi...,https://en.wikipedia.org/wiki/WebPlatform.org


https://en.wikipedia.org/wiki/Richard_Wiseman


In [28]:
df.to_csv('data.csv', index=False)

In [29]:
read_df = pd.read_csv("data.csv", index_col=False)
display(read_df)

,Title,Description,Link
0,Web scraping,"Web scraping, web harvesting, or web data extr...",https://en.wikipedia.org/wiki/Web_scraping
1,Wrapper (data mining),Wrapper in data mining is a procedure that ext...,https://en.wikipedia.org/wiki/Wrapper_(data_mi...
2,Microformat,Microformats (μF)[note 1] are a set of defined...,https://en.wikipedia.org/wiki/Microformat
3,Common Logic,Common Logic (CL) is a framework for a family ...,https://en.wikipedia.org/wiki/Common_Logic
4,ISLISP,ISLISP (also capitalized as ISLisp) is a progr...,https://en.wikipedia.org/wiki/ISLISP
...,...,...,...
22121,Filter (software),A filter is a computer program or subroutine t...,https://en.wikipedia.org/wiki/Filter_(Unix)
22122,Name collision,"In computer programming, a name collision is t...",https://en.wikipedia.org/wiki/Name_collision
22123,WebXR,WebXR Device API is a Web application programm...,https://en.wikipedia.org/wiki/WebXR
22124,WebPlatform.org,WebPlatform.org (or WebPlatform) was a communi...,https://en.wikipedia.org/wiki/WebPlatform.org
